导入所需要的库（llama_cpp库如果直接安装失败可以尝试下载对应的whl包进行安装，下载地址：https://abetlen.github.io/llama-cpp-python/whl/cpu/llama-cpp-python/ ）

In [3]:
from llama_cpp import Llama
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

本地部署大模型：需在huggingface上下载模型文件，下载地址：https://huggingface.co/Triangle104/Qwen2.5-3B-Instruct-abliterated-Q4_K_M-GGUF/tree/main

In [4]:
llm = Llama(model_path=".\qwen2.5-3b-instruct-abliterated-q4_k_m.gguf", verbose=False,)

llama_new_context_with_model: n_ctx_per_seq (512) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


一个模型对话的示例：

In [5]:
prompt = "Where is Southeast University located?"
output = llm.create_chat_completion(
    messages=[{
        "role": "user",
        "content": prompt
    }],
    max_tokens=200,
    stream=False,
)
answer = output["choices"][0]["message"]["content"]
print(answer)

Southeast University is located in Nanjing, Jiangsu Province, China.


请比较一下提示词工程对大模型输出的影响,
下面是第三次实验的机器翻译任务（可以通过对translate函数进行修改加入一些提示词工程），也可以尝试其他任务

In [51]:
def translate(english_sentence):
    instruntion = "Translate the following sentence to Japanese: "

    output = llm.create_chat_completion(
        messages=[{
            "role": "user",
            "content": instruntion+" "+english_sentence
        }],
        max_tokens=200,
        stream=False,
    )
    answer = output["choices"][0]["message"]["content"]
    return answer

In [52]:
chinese_dev_list = []
english_dev_list = []
with open("../exp3/data/dev.txt","r") as f:
    for line in f.readlines():
        english,chinese = line.strip().split("\t")
        chinese_dev_list.append(chinese)
        english_dev_list.append(english)

references = []
candidates = []
for i in range(10):
    e_s = english_dev_list[i]
    c_s = chinese_dev_list[i]
    answer = translate(e_s)
    references.append([c_s])
    candidates.append(answer)
    print('reference:', c_s)
    print('candidate:', answer)
    
smoothie = SmoothingFunction().method1
corpus_score = corpus_bleu(
    references,
    candidates,
    smoothing_function=smoothie
)
print(corpus_score)

reference: 照顾好自己。
candidate: The translated sentence for "Take care." in Japanese is "お気をください。" (O kikasueru kudasai. )
reference: 在这等着。
candidate: The translated sentence for "Wait here." in Japanese is "ここに待ってください." (Koko ni tai sure kudasai. )
reference: 做得好！
candidate: Well done! 可能な限り日本語で返答しますので、次の質問をお待ちしております。とします。次の質問を待ちます! と訳すと、次の日本語は「次の質問をお待ちしております!」になります。したがって、「Well done!」は「とても良い!」または「素晴らしい!」という意味になります。「とても良い!」の日本語訳は「とても良い!」で、「素晴らしい!」の日本語訳は「素晴らしい!」になります。したがって、「Well done!」の日本語訳は「とても良い!」または「素晴らしい!」になります。
reference: 他向我施压。
candidate: The translation of "He pressured me" to Japanese is "彼は私を圧迫しました" (Kare wa watashi o kakushin-shimashita).
reference: 他努力學習。
candidate: The translated sentence in Japanese is:  他は努力を惜しまなかった。 (Kata wa kouzai o sakina nakatta. )
reference: 他企图说谎。
candidate: The translated sentence is: 他 tends to lie. (Kano de ta tai ri.)
reference: 他很容易觉得累。
candidate: The translated sentence for "He tires easily." in Japanese is:

「彼は疲れやすいです。」
(Kano wa tsuyaku sutari d